<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8


127 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-03-27 06:57:20--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?DhGaN760plgcNKFHUym1uOkAu7Ak7RD04QJNuBYsqd78O1MarQAZC-hlmRjoFRaTbAzXhs0RgJJxunjb_6vunpmeIVSwdoMRPL2l3Gqc-M-LA79Tn0S9tdQ5ZKZdm34tTYDe8T5MojEZxw70GSRyMVZ_LEyF7567CUffUITXwovAEJI_QMUl59S1Mw9QKfZLpD1OVR9b_Rw7jAkdxzNhhojp-g [following]
--2019-03-27 06:57:21--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?DhGaN760plgcNKFHUym1u

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zxtta07e
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbw6avyj/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [132]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <float.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 10000000
#define N 512

#define e 2.7182818

#define blockZSize 1

#define blockXSize 64
#define blockYSize 64

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[6 * 6];

  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
      int dAI = iBegin - blockZSize + i;
      int dAJ = jBegin - blockZSize + j;
      if(dAI < 0 || dAI >= extN || dAJ < 0 || dAJ >= extN) {
          helper[i * N1 + j] = FLT_MAX;
      }
      else {
          helper[i * N1 + j] = matrix[extN * dAI + dAJ];
      }
		}
	}
  
  int iSplit = iBegin - blockZSize;
  int jSplit = jBegin - blockZSize;
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
		  for (int j = k; j < N2 - k; j += 1) {
        int b = N1 * i + j;
        if(helper[b + N1] != FLT_MAX && helper[b - N1] != FLT_MAX && helper[b + 1] != FLT_MAX && helper[b - 1] != FLT_MAX) {
            helper[b] = 0.25 * (
                helper[b + N1]
              + helper[b - N1]
              + helper[b + 1]
              + helper[b - 1] 
              - h2 * f(iSplit + i, jSplit + j)
            );
        }
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);
    
  for (int i = iBegin; i < iBegin + tileXSize; i++) {
		for (int j = jBegin; j < jBegin + tileYSize; j += 1) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i - blockZSize + 1, j - blockZSize + 1)
      );
      helper[helperIndex] =	matrix[extN * i + j];
		}
	}
    
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int i = 0; i < row; i++) {
      matrix[0 * row + i] = u_t((i) * h);
      matrix[(row - 1) * row + i] = u_d((i) * h);
  }

  for(int j = 0; j < col; j++) {
      matrix[j * col + 0] = u_l((j) * h);
      matrix[j * col + row - 1] = u_r((j) * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = 1; i < extN - 1; i++) {
		for (int j = 1; j < extN - 1; j++) {
      float fCurrent = func(i, j);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
    ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;

	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

    if(i%100 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.1) break;

    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

    myfile.close();
	return 0;
}

' i: 510j: 510 Difference: 7.30318iter: 0 i: 480j: 480 Difference: 6.39734iter: 100 i: 468j: 468 Difference: 6.06692iter: 200 i: 460j: 460 Difference: 5.83344iter: 300 i: 453j: 453 Difference: 5.64914iter: 400 i: 446j: 446 Difference: 5.49367iter: 500 i: 441j: 441 Difference: 5.36035iter: 600 i: 437j: 437 Difference: 5.24131iter: 700 i: 432j: 432 Difference: 5.13438iter: 800 i: 428j: 428 Difference: 5.03706iter: 900 i: 424j: 424 Difference: 4.94762iter: 1000 i: 421j: 421 Difference: 4.86483iter: 1100 i: 417j: 417 Difference: 4.78775iter: 1200 i: 413j: 413 Difference: 4.7151iter: 1300 i: 411j: 411 Difference: 4.64679iter: 1400 i: 408j: 408 Difference: 4.5825iter: 1500 i: 405j: 405 Difference: 4.52171iter: 1600 i: 402j: 402 Difference: 4.46346iter: 1700 i: 400j: 400 Difference: 4.40804iter: 1800 i: 397j: 397 Difference: 4.35545iter: 1900 i: 394j: 394 Difference: 4.30456iter: 2000 i: 392j: 392 Difference: 4.25606iter: 2100 i: 389j: 389 Difference: 4.20936iter: 2200 i: 388j: 388 Difference

In [133]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <float.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 10000000
#define N 512

#define e 2.7182818

#define blockZSize 1

#define blockXSize 64
#define blockYSize 64

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  int N1 = tileXSize + 2;
  int N2 = tileYSize + 2;
  
	float helper[6 * 6];

  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
      int dAI = iBegin - blockZSize + i;
      int dAJ = jBegin - blockZSize + j;
      helper[i * N1 + j] = matrix[extN * dAI + dAJ];
		}
	}
  
  int iSplit = iBegin - 1;
  int jSplit = jBegin - 1;
  
  for (int i = iBegin; i < iBegin + tileXSize; i++) {
		for (int j = jBegin; j < jBegin + tileYSize; j += 1) {
      int helperIndex = N1 * (i - iBegin + 1) + j - jBegin + 1;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i, j)
      );
      helper[helperIndex] =	matrix[extN * i + j];
		}
	}
    
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int i = 0; i < row; i++) {
      matrix[0 * row + i] = u_t((i) * h);
      matrix[(row - 1) * row + i] = u_d((i) * h);
  }

  for(int j = 0; j < col; j++) {
      matrix[j * col + 0] = u_l((j) * h);
      matrix[j * col + row - 1] = u_r((j) * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = 1; i < extN - 1; i++) {
		for (int j = 1; j < extN - 1; j++) {
      float fCurrent = func(i, j);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
    ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;

	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

    if(i%100 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.1) break;

    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

    myfile.close();
	return 0;
}

' i: 510j: 510 Difference: 7.30318iter: 0 i: 480j: 480 Difference: 6.39734iter: 100 i: 468j: 468 Difference: 6.06697iter: 200 i: 460j: 460 Difference: 5.8336iter: 300 i: 453j: 453 Difference: 5.64934iter: 400 i: 446j: 446 Difference: 5.49388iter: 500 i: 441j: 441 Difference: 5.36051iter: 600 i: 437j: 437 Difference: 5.24145iter: 700 i: 432j: 432 Difference: 5.13451iter: 800 i: 428j: 428 Difference: 5.03719iter: 900 i: 424j: 424 Difference: 4.94774iter: 1000 i: 421j: 421 Difference: 4.86495iter: 1100 i: 417j: 417 Difference: 4.78787iter: 1200 i: 413j: 413 Difference: 4.71521iter: 1300 i: 411j: 411 Difference: 4.6469iter: 1400 i: 408j: 408 Difference: 4.58261iter: 1500 i: 405j: 405 Difference: 4.52181iter: 1600 i: 402j: 402 Difference: 4.46354iter: 1700 i: 400j: 400 Difference: 4.40811iter: 1800 i: 397j: 397 Difference: 4.35551iter: 1900 i: 394j: 394 Difference: 4.30461iter: 2000 i: 392j: 392 Difference: 4.25609iter: 2100 i: 389j: 389 Difference: 4.20939iter: 2200 i: 388j: 388 Difference

In [75]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 10 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(14), end='')
  i += 1



             1       1.11752       1.24885       1.39561       1.55962       1.74291       1.94773       2.17663       2.43243       2.71828             
       1.11752             0             0             0             0             0             0             0             0       3.03773             
       1.24885             0             0             0             0             0             0             0             0       3.39472             
       1.39561             0             0             0             0             0             0             0             0       3.79367             
       1.55962             0             0             0             0             0             0             0             0        4.2395             
       1.74291             0             0             0             0             0             0             0             0       4.73772             
       1.94773             0             0             0             0    